# 10: Recommender Systems

The hand-in exercise for this topic is Exercise 1 from the notebook “Exercises in Recommender systems.ipynb"

---

## Exercise 1

Using the "Coursera Courses Dataset 2021" available at kaggle ([https://www.kaggle.com/datasets/khusheekapoor/coursera-courses-dataset-2021](https://www.kaggle.com/datasets/khusheekapoor/coursera-courses-dataset-2021)) or on moodle, to do the following:

1. Create a Content-based filtering recommender system based on the Course Descriptions.
2. Create a Content-based filtering recommender system based on the Skills.

Using the "Book Recommendation Dataset" available at kaggle ([https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset)) or on moodle, to do the following:

3. Load in the `Ratings.csv` file (on moodle, it is called `Books_Ratings.csv`). Group by `User-ID` and sort by `Book-Rating` in descending order to get the users who rated most books. Filter the rating data to only contain the 200 users that rated most books.
4. Create a Collaborative filtering recommender system based on the user ratings from 3 together with the `Books.csv` dataset.

---

In [33]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505177 sha256=38fea4f90a6f0926f5a317ffd5bb0379638dcda65f75480bb794f84236346c26
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [32]:
# Lib imports for the notebook
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict

ModuleNotFoundError: No module named 'surprise'

In [17]:
# Only for while working in google colab, can be removed again using jupyter
# from google.colab import files
# uploaded = files.upload()

---

### Coursera Courses Dataset 2021
1. Create a Content-based filtering recommender system based on the Course Descriptions.
2. Create a Content-based filtering recommender system based on the Skills.

Some EDA to prepare and look at the structure of the data

In [18]:
coursera_data = pd.read_csv('Coursera.csv')

In [20]:
coursera_data.head(10)

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...
5,Building Test Automation Framework using Selen...,Coursera Project Network,Beginner,4.7,https://www.coursera.org/learn/building-test-a...,Selenium is one of the most widely used functi...,maintenance test case test automation scree...
6,Doing Business in China Capstone,The Chinese University of Hong Kong,Advanced,3.3,https://www.coursera.org/learn/doing-business-...,Doing Business in China Capstone enables you t...,marketing plan Planning Marketing consumpti...
7,"Programming Languages, Part A",University of Washington,Intermediate,4.9,https://www.coursera.org/learn/programming-lan...,This course is an introduction to the basic co...,inference ml (programming language) higher-o...
8,The Roles and Responsibilities of Nonprofit Bo...,The State University of New York,Intermediate,4.3,https://www.coursera.org/learn/nonprofit-gov-2,This course provides a more in-depth look at t...,Planning Peer Review fundraising strategic ...
9,Business Russian Communication. Part 3,Saint Petersburg State University,Intermediate,Not Calibrated,https://www.coursera.org/learn/business-russia...,Russian is considered to be one of the most di...,Russian market (economics) tax exemption co...


In [21]:
coursera_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


---

### 1. Create a Content-based filtering recommender system based on the Course Descriptions.

### Recommendation based on similariry in 'Description' feature

* Write the name of a course from the data you like and the function will recommend courses with similar term frequency

* I like the course "Retrieve Data using Single-Table SQL Queries" so im going to look for 5 similar ones

In [27]:
# Use TFIDF vectorization to calc term frequency and match documents with similar vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english') # skip common english words so they dont affect the search
# Fit and transform the Course Descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(coursera_data['Course Description'])
# we need the cosine similarity matrix, to find similarities between input description vs lookup
cosine_similar_descriptions = cosine_similarity(tfidf_matrix, tfidf_matrix)
# Func for recommending similar courses to the search, based on simiar term frequency
def recommend_courses_based_on_desc(course_title, num_recommendations):
    if course_title not in coursera_data['Course Name'].values:
        return "Course not found." # if the course dont exist in the data

    idx = coursera_data[coursera_data['Course Name'] == course_title].index[0]
    similarity_scores = list(enumerate(cosine_similar_descriptions[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:num_recommendations+1]
    recommended_courses = [coursera_data.iloc[i[0]]['Course Name'] for i in similarity_scores]
    return recommended_courses

# Make a search and get a recommendation on the course i like
recommend_courses_based_on_desc("Retrieve Data using Single-Table SQL Queries", 5) # I want 5 recommendationss

['Creating Database Tables with SQL',
 'Querying Databases Using SQL SELECT statement',
 'Advanced Relational Database and SQL',
 'Create Relational Database Tables Using SQLiteStudio',
 'Intermediate Relational Database and SQL']

---

### 2. Create a Content-based filtering recommender system based on the Skills

* We can use the same syntax as before, but instead targeting the 'Skills' feature

* Write the name of a course from the data you like and the function will recommend courses with similar term frequency

* I like the course "Retrieve Data using Single-Table SQL Queries" so im going to look for 5 similar ones

In [28]:
# Use TFIDF vectorization to calc term frequency and match documents with similar vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english') # skip common english words so they dont affect the search
# Fit and transform the Course Descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(coursera_data['Skills'])
# we need the cosine similarity matrix, to find similarities between input description vs lookup
cosine_similar_descriptions = cosine_similarity(tfidf_matrix, tfidf_matrix)
# Func for recommending similar courses to the search, based on simiar term frequency
def recommend_courses_based_on_desc(course_title, num_recommendations):
    if course_title not in coursera_data['Course Name'].values:
        return "Course not found." # if the course dont exist in the data

    idx = coursera_data[coursera_data['Course Name'] == course_title].index[0]
    similarity_scores = list(enumerate(cosine_similar_descriptions[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:num_recommendations+1]
    recommended_courses = [coursera_data.iloc[i[0]]['Course Name'] for i in similarity_scores]
    return recommended_courses

# Make a search and get a recommendation on the course i like
recommend_courses_based_on_desc("Retrieve Data using Single-Table SQL Queries", 5) # I want 5 recommendationss as defined as parameter for the func

['Manipulating Data with SQL',
 'Creating Database Tables with SQL',
 'Managing Big Data with MySQL',
 'Create Relational Database Tables Using SQLiteStudio',
 'Retrieve Data with Multiple-Table SQL Queries']

---

### Book Recommendation Dataset
3. Load in the Ratings.csv file (on moodle, it is called Books_Ratings.csv). Group by User-ID and sort by Book-Rating in descending order to get the users who rated most books. Filter the rating data to only contain the 200 users that rated most books.
4. Create a Collaborative filtering recommender system based on the user ratings from 3 together with the Books.csv dataset.

### First steps EDA

In [44]:
books_data = pd.read_csv('Books_Ratings.csv')

In [30]:
books_data.head(10)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
6,276736,3257224281,8
7,276737,0600570967,6
8,276744,038550120X,7
9,276745,342310538,10


In [29]:
books_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


---

### 3. Load in the Ratings.csv file (on moodle, it is called Books_Ratings.csv). Group by User-ID and sort by Book-Rating in descending order to get the users who rated most books. Filter the rating data to only contain the 200 users that rated most books.

In [45]:
# Counts for rating pr user
user_rating_count = books_data.groupby("User-ID").size().reset_index(name="User-Rating-Count")
# Quick look to see if it looks correct
user_rating_count

,User-ID,User-Rating-Count
0,2,1
1,7,1
2,8,18
3,9,3
4,10,2
...,...,...
105278,278846,2
105279,278849,4
105280,278851,23
105281,278852,1


In [46]:
# Sort the grouped data in descending order
# Filter the data into a new dataframe, which will only include the 200 users with most ratings in descending order
top_users = user_rating_count.sort_values(by="User-Rating-Count", ascending=False).head(200)
# Quick look to see if it looks correct
top_users

,User-ID,User-Rating-Count
4213,11676,13602
74815,198711,7550
58113,153662,6109
37356,98391,5891
13576,35859,5850
...,...,...
99371,262998,679
79124,210035,678
50591,133747,677
40296,106225,677


In [54]:
filtered_books_data = books_data[books_data["User-ID"].isin(top_users["User-ID"])]
# This will now only show top 200 rating users and all the books they have rated
filtered_books_data['User-ID'].nunique()

200

---

### 4. Create a Collaborative filtering recommender system based on the user ratings from 3 together with the Books.csv dataset.

---